# Biosimulators compatibility

## set-up

In [1]:
#!/usr/bin/env python

'''
use pymetadata module to create a minimal valid combine archive
using LEMS_NML2_Ex9_FN.sbml and LEMS_NML2_Ex9_FN.sedml
'''

import sys
sys.path.append("..")
import utils
import os
import pandas as pd
from IPython.display import display_markdown
import shutil

sbml_filepath = 'LEMS_NML2_Ex9_FN.sbml'
sedml_filepath = 'LEMS_NML2_Ex9_FN_missing_xmlns.sedml' #xmlns:sbml missing

engines = utils.engines
types_dict = utils.types_dict

cwd = os.getcwd()
print(cwd)

c:\Users\prins\GitHub\SBMLShowcase\SBML


## test compatibility biosimulators remotely
see https://github.com/OpenSourceBrain/SBMLShowcase/issues/32

In [27]:
from pyneuroml import biosimulations

# get the version of the engine
engine = 'tellurium'
engine_version = biosimulations.get_simulator_versions(engine)

#put the sedml and sbml into a combine archive
omex_filepath = utils.create_omex(sedml_filepath,sbml_filepath)
omex_file_name = os.path.basename(omex_filepath)

sim_dict = {
              "name": "test",
              "simulator": engine,
              "simulatorVersion": engine_version[engine][-1],
              "cpus": 1,
              "memory": 8,
              "maxTime": 20,
              "envVars": [],
              "purpose": "academic",
              "email": "ucbtrin@ucl.ac.uk",
            }

download_link, result = biosimulations.submit_simulation_archive(\
    archive_file=omex_file_name,\
    sim_dict=sim_dict)


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'tellurium', 'simulatorVersion': '2.2.8', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': 'ucbtrin@ucl.ac.uk'}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"tellurium","simulatorVersion":"2.2.8","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"ucbtrin@ucl.ac.uk","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"tellurium","simulatorVersion":"2.2.8","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"ucbtrin@ucl.ac.uk","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"tellur

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 668666cf6b7c343121b0ac70
View: https://api.biosimulations.org/runs/668666cf6b7c343121b0ac70
Downloads: https://api.biosimulations.org/results/668666cf6b7c343121b0ac70/download
Logs: https://api.biosimulations.org/logs/668666cf6b7c343121b0ac70?includeOutput=true
Download completed.


In [28]:
filepath_results = utils.download_file_from_link(download_link)
extract_dir = os.path.join(cwd, 'results')
# fetch yml log file from extract_dir
yml_log =  os.path.join(extract_dir, 'log.yml')
print(yml_log)

shutil.unpack_archive(filepath_results, extract_dir=extract_dir)
os.remove(filepath_results)


Download completed.
c:\Users\prins\GitHub\SBMLShowcase\SBML\results\log.yml


## test compatibility biosimulators locally using docker
see https://github.com/OpenSourceBrain/SBMLShowcase/issues/31


In [7]:
engine_dict = {}

output_folder = 'output'

for e in engines.keys():
    print('Running ' + e)
    output_dir = os.path.abspath(os.path.join(output_folder, e))
    try:
        record = utils.run_biosimulators_docker(e, sedml_filepath, sbml_filepath, output_dir=output_dir)
        engine_dict[e] = record
    except Exception as error:
        error_message = str(error)
        print(f"Error occurred while running {e}")
        engine_dict[e] = error_message
        continue

file_paths = utils.find_files(output_folder, '.pdf')
utils.move_d1_files(file_paths, 'd1_plots')
shutil.rmtree(output_folder)

Running amici
Running brian2
Running bionetgen
Running boolnet
Running cbmpy
Running cobrapy
Running copasi
Running gillespy2
Running ginsim
Running libsbmlsim
Running masspy
Running netpyne
Running neuron
Running opencor
Running pyneuroml
Running pysces
Running rbapy
Running smoldyn
Running tellurium
Running vcell
Running xpp
d1_plots\amici_d1.pdf
d1_plots\bionetgen_d1.pdf
d1_plots\boolnet_d1.pdf
d1_plots\cbmpy_d1.pdf
d1_plots\cobrapy_d1.pdf
d1_plots\copasi_d1.pdf
d1_plots\gillespy2_d1.pdf
d1_plots\ginsim_d1.pdf
d1_plots\libsbmlsim_d1.pdf
d1_plots\masspy_d1.pdf
d1_plots\pysces_d1.pdf
d1_plots\rbapy_d1.pdf
d1_plots\tellurium_d1.pdf


In [8]:
# Create a table of the results
results_table = pd.DataFrame.from_dict(engine_dict).T
results_table.columns = ['pass/FAIL', 'Error']
results_table.index.name = 'Engine'
results_table.reset_index(inplace=True)

results_table['Error'] = results_table.apply(lambda x: None if x['pass/FAIL'] == x['Error'] else x['Error'], axis=1)
results_table['pass/FAIL'] = results_table['pass/FAIL'].replace('other', 'FAIL')
# # results_table['pass/FAIL'] = results_table['pass/FAIL'].apply(lambda x: x if x == 'FAIL' else x)

# for non empty cells in error col print error message

results_table['Error'] = results_table['Error'].apply(lambda x: utils.ansi_to_html(x))
results_table['Error'] = results_table['Error'].apply(lambda x: utils.collapsible_content(x))

# compatibility_message
results_table['Compatibility'] = results_table['Engine'].apply(lambda x: utils.check_file_compatibility_test(x, types_dict, sbml_filepath, sedml_filepath))
results_table['Compatibility'] = results_table['Compatibility'].apply(lambda x: utils.collapsible_content(x[1], title=x[0]))
results_table['pass/FAIL'] = results_table['pass/FAIL'].apply(lambda x: f'<span style="color:darkred;">{x}</span>' if x == 'FAIL' else x)
results_table['Compatibility'] = results_table['Compatibility'].apply(lambda x: f'<span style="color:darkred;">{x}</span>' if 'FAIL' in x else x)

# d1 plot clickable link
results_table['d1'] = results_table['Engine'].apply(lambda x: utils.d1_plots_dict(engines, 'd1_plots').get(x, None))
results_table['d1'] = results_table['d1'].apply(lambda x: utils.create_hyperlink(x))

results_md_table = results_table.to_markdown(index=False)
display_markdown(results_md_table, raw=True)


| Engine     | pass/FAIL                                | Error                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Compatibility                                                                                                                                                                                                                                                              | d1                                                         |
|:-----------|:-----------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------|
| amici      | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with amici</details>                                                                                                                       | <a href="d1_plots\amici_d1.pdf">amici_d1.pdf</a>           |
| brian2     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with brian2</details>                                                                                                                      |                                                            |
| bionetgen  | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp149263` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in BNGL format (e.g., `sed:model/@language` must match `^urn:sedml:language:bngl(\.$)` such as `urn:sedml:language:bngl`).<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with bionetgen. The following file types will be compatible ['BNGL', 'SED-ML']</details></span>            | <a href="d1_plots\bionetgen_d1.pdf">bionetgen_d1.pdf</a>   |
| boolnet    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp151894` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - Number of points (20000) must be equal to the difference between the output end (200.0) and start times (0.0).<br>R[write to console]: Warning message:<br><br>R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :<br>R[write to console]: <br> <br>R[write to console]:  library â/usr/lib/R/site-libraryâ contains no packages<br><br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with boolnet. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>         | <a href="d1_plots\boolnet_d1.pdf">boolnet_d1.pdf</a>       |
| cbmpy      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><br>INFO: No xlwt module available, Excel spreadsheet creation disabled<br><br><br><br>No module named 'cplex'<br><br><br><br>CPLEX not available<br><br>*****<br>Using GLPK<br>*****<br><br><br>INFO: No xlrd module available, Excel spreadsheet reading disabled<br><br>CBMPy environment<br>******************<br>Revision: r689<br><br><br>***********************************************************************<br>* Welcome to CBMPy (0.7.25) - PySCeS Constraint Based Modelling       *<br>*                http://cbmpy.sourceforge.net                         *<br>* Copyright(C) Brett G. Olivier 2014 - 2019                           *<br>* Dept. of Systems Bioinformatics                                     *<br>* Vrije Universiteit Amsterdam, Amsterdam, The Netherlands            *<br>* CBMPy is developed as part of the BeBasic MetaToolKit Project       *<br>* Distributed under the GNU GPL v 3.0 licence, see                    *<br>* LICENCE (supplied with this release) for details                    *<br>***********************************************************************<br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp369389` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with cbmpy</details>                                                                                                                       | <a href="d1_plots\cbmpy_d1.pdf">cbmpy_d1.pdf</a>           |
| cobrapy    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp31836` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with cobrapy</details>                                                                                                                     | <a href="d1_plots\cobrapy_d1.pdf">cobrapy_d1.pdf</a>       |
| copasi     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp987364` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    could not convert string to float: 'Values[V]'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with copasi</details>                                                                                                                      | <a href="d1_plots\copasi_d1.pdf">copasi_d1.pdf</a>         |
| gillespy2  | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with gillespy2</details>                                                                                                                   | <a href="d1_plots\gillespy2_d1.pdf">gillespy2_d1.pdf</a>   |
| ginsim     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp382733` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - The interval between the output start and time time must be an integer multiple of the number of steps, not `0.01`:<br>          Output start time: 0.0<br>          Output end time: 200.0<br>          Number of steps: 20000<br></details> | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with ginsim. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>          | <a href="d1_plots\ginsim_d1.pdf">ginsim_d1.pdf</a>         |
| libsbmlsim | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with libsbmlsim</details>                                                                                                                  | <a href="d1_plots\libsbmlsim_d1.pdf">libsbmlsim_d1.pdf</a> |
| masspy     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31 [1;31m<br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31 [1;31m<br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp750289` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `mass.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .<br>    	`(model, errors) = validate_sbml_model(filename)`<br>    If the model is valid and cannot be read please open an issue at https://github.com/SBRG/masspy/issues .<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with masspy</details>                                                                                                                      | <a href="d1_plots\masspy_d1.pdf">masspy_d1.pdf</a>         |
| netpyne    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with netpyne</details>                                                                                                                     |                                                            |
| neuron     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with neuron. The following file types will be compatible ['SED-ML', 'NeuroML', 'LEMS']</details></span>    |                                                            |
| opencor    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with opencor. The following file types will be compatible ['CellML', 'SED-ML']</details></span>            |                                                            |
| pyneuroml  | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with pyneuroml. The following file types will be compatible ['SED-ML', 'NeuroML', 'LEMS']</details></span> |                                                            |
| pysces     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>Could not find GLIMDA.<br>/usr/local/lib/python3.10/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.10/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp757728` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Model at /tmp/tmp0b0_er9p/./LEMS_NML2_Ex9_FN.sbml could not be imported:<br>      <br>      File /tmp/tmp0b0_er9p/./LEMS_NML2_Ex9_FN.sbml.xml does not exist<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with pysces</details>                                                                                                                      | <a href="d1_plots\pysces_d1.pdf">pysces_d1.pdf</a>         |
| rbapy      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp987912` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in RBA format (e.g., `sed:model/@language` must match `^urn:sedml:language:rba(\.$)` such as `urn:sedml:language:rba`).<br></details>                 | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with rbapy. The following file types will be compatible ['RBApy', 'SED-ML']</details></span>               | <a href="d1_plots\rbapy_d1.pdf">rbapy_d1.pdf</a>           |
| smoldyn    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>[Errno 2] No such file or directory: '/tmp/tmpmu42eki7/model.txt'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | <span style="color:darkred;"><details><summary>FAIL</summary>smoldyn compatible file types unknown.</details></span>                                                                                                                                                       |                                                            |
| tellurium  | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. These are compatible with tellurium</details>                                                                                                                   | <a href="d1_plots\tellurium_d1.pdf">tellurium_d1.pdf</a>   |
| vcell      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/vcell' returned non-zero exit status 1: b'  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  PROCESSING_SEDML(preProcessDoc)  **** Error: SED-ML processing for /tmp/vcell_temp_4d26fa44-2457-4a00-b0d1-d8bf8a3a541412142204403474173125/tmp605892 failed with error: \n  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  SIMULATIONS_RUN(runSimulations)  **** Error: \n  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  PROCESSING_SIMULATION_OUTPUTS(processOutputs)  **** Error: Error processing outputs\n'```</details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | <span style="color:darkred;"><details><summary>FAIL</summary>vcell compatible file types unknown.</details></span>                                                                                                                                                         |                                                            |
| xpp        | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest the input file types are '['SED-ML', 'SBML']'. Tese are not compatible with xpp. The following file types will be compatible ['XPP', 'SED-ML']</details></span>                   |                                                            |

In [9]:
# save results_md_table
with open('results_compatibility_biosimulators.md', 'w', encoding='utf-8') as f:
    f.write(results_md_table)